# --- Day 18: Duet ---

http://adventofcode.com/2017/day/18

You discover a tablet containing some strange assembly code labeled simply "Duet". Rather than bother the sound card with it, you decide to run the code yourself. Unfortunately, you don't see any documentation, so you're left to figure out what the instructions mean on your own.

It seems like the assembly is meant to operate on a set of registers that are each named with a single letter and that can each hold a single integer. You suppose each register should start with a value of 0.

There aren't that many instructions, so it shouldn't be hard to figure out what they do. Here's what you determine:

- snd X plays a sound with a frequency equal to the value of X.
- set X Y sets register X to the value of Y.
- add X Y increases register X by the value of Y.
- mul X Y sets register X to the result of multiplying the value contained in register X by the value of Y.
- mod X Y sets register X to the remainder of dividing the value contained in register X by the value of Y (that is, it sets X to the result of X modulo Y).
- rcv X recovers the frequency of the last sound played, but only when the value of X is not zero. (If it is zero, the command does nothing.)
- jgz X Y jumps with an offset of the value of Y, but only if the value of X is greater than zero. (An offset of 2 skips the next instruction, an offset of -1 jumps to the previous instruction, and so on.)

Many of the instructions can take either a register (a single letter) or a number. The value of a register is the integer it contains; the value of a number is that number.

After each jump instruction, the program continues with the instruction to which the jump jumped. After any other instruction, the program continues with the next instruction. Continuing (or jumping) off either end of the program terminates it.

For example:

```
set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2
```

- The first four instructions set a to 1, add 2 to it, square it, and then set it to itself modulo 5, resulting in a value of 4.
- Then, a sound with frequency 4 (the value of a) is played.
- After that, a is set to 0, causing the subsequent rcv and jgz instructions to both be skipped (rcv because a is 0, and jgz because a is not greater than 0).
- Finally, a is set to 1, causing the next jgz instruction to activate, jumping back two instructions to another jump, which jumps again to the rcv, which ultimately triggers the recover operation.

At the time the recover operation is executed, the frequency of the last sound played is 4.

**What is the value of the recovered frequency** (the value of the most recently played sound) the first time a rcv instruction is executed with a non-zero value?

## Part one solution:

This is similar to an earlier solution, the key thing is knowing the collections and using defaultdic, and parsing the incoming instructions:

In [13]:
# the puzzle input
with open('puzzle_inputs/day18_input.txt') as f:
    data = f.read().strip().split("\n")
puzzle_input = [line for line in data]
puzzle_input[:10]

['set i 31',
 'set a 1',
 'mul p 17',
 'jgz p p',
 'mul a 2',
 'add i -1',
 'jgz i -2',
 'add a -1',
 'set i 127',
 'set p 735']

In [15]:
from collections import defaultdict

def part_one(instructions, verbose=False):
    """goes through instructions and perfoms them as per part one"""
    
    registers = defaultdict(int)
    sounds = []
    i = 0
    total_instructions = len(instructions)
    
    while i >=0 and i < total_instructions:
        
        # first, parse instruction
        f = instructions[i].split()
        if verbose: print(i, f)
        func = f[0]
        
        try:
            x = int(f[1])
        except:
            x = f[1]

        if len(f) == 3:
            try:
                y = int(f[2])
            except:
                y = f[2]
        
        if func == "rcv":
            if verbose: print(registers)
            print("part one answer is:", sounds[-1])
            break
        
        # now to run the func code
        
        if func == "snd":
            sounds.append(registers[x])
        elif func == "set":
            y = registers[y] if type(y) is str else y
            registers[x] = y
        elif func == "add":
            y = registers[y] if type(y) is str else y
            registers[x] += y
        elif func == "mul":
            y = registers[y] if type(y) is str else y
            registers[x] = registers[x] * y
        elif func == "mod":
            y = registers[y] if type(y) is str else y
            registers[x] = registers[x] % y
        elif func == "rcv":
            x = registers[x] if type(x) is str else x
            if x != 0:
                sounds.append(sounds[-1])
        elif func == "jgz":
            x = registers[x] if type(x) is str else x
            if x > 0:
                i += registers[y] if type(y) is str else y
                if verbose: print(f"jumping to {i}")
                # now to skip the i += step below
                continue
                
        i += 1

part_one(puzzle_input, verbose=False)

part one answer is: 8600


`8600` was the right answer for my puzzle input! yay!

# --- Part Two ---

As you congratulate yourself for a job well done, you notice that the documentation has been on the back of the tablet this entire time. While you actually got most of the instructions correct, there are a few key differences. This assembly code isn't about sound at all - it's meant to be run twice at the same time.

Each running copy of the program has its own set of registers and follows the code independently - in fact, the programs don't even necessarily run at the same speed. To coordinate, they use the send (snd) and receive (rcv) instructions:

- `snd X` sends the value of X to the other program. These values wait in a queue until that program is ready to receive them. Each program has its own message queue, so a program can never receive a message it sent.
- `rcv X` receives the next value and stores it in register X. If no values are in the queue, the program waits for a value to be sent to it. Programs do not continue to the next instruction until they have received a value. Values are received in the order they are sent.

Each program also has its own program ID (one 0 and the other 1); the register p should begin with this value.

For example:

```
snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d```

Both programs begin by sending three values to the other. Program 0 sends 1, 2, 0; program 1 sends 1, 2, 1. Then, each program receives a value (both 1) and stores it in a, receives another value (both 2) and stores it in b, and then each receives the program ID of the other program (program 0 receives 1; program 1 receives 0) and stores it in c. Each program now sees a different value in its own copy of register c.

Finally, both programs try to rcv a fourth time, but no data is waiting for either of them, and they reach a deadlock. When this happens, both programs terminate.

It should be noted that it would be equally valid for the programs to run at different speeds; for example, program 0 might have sent all three values and then stopped at the first rcv before program 1 executed even its first instruction.

Once both of your programs have terminated (regardless of what caused them to do so), **how many times did program 1 send a value?**

This seems like a good reason use multi-threading/processing, probably using Process, Queue, Pipe from mp, but since I'm using jupyter notebook running in termux on a chromebook, multiprocessing doesn't work:

> This platform lacks a functioning sem_open implementation, therefore, the required synchronization primitives needed will not function, see issue 3770.

so trying threading:

In [16]:
from threading import Thread, Lock
from queue import Queue
from collections import defaultdict

lock = Lock()

def do_ins_threaded(instructions, progam_id, inqueue, outqueue, lock=lock, verbose=False):
    """goes through instructions and perfoms them as per part one"""
    
    registers = defaultdict(int)
    registers["p"] = progam_id
    
    total_instructions = len(instructions)
    sent_msgs = 0
    rcv_msgs = 0
    i = 0
    
    while i >=0 and i < total_instructions:
        # first, parse instruction
        f = instructions[i].split()
        #if verbose: print(progam_id, i, total_instructions)
        func = f[0]
        
        try:
            x = int(f[1])
        except:
            x = f[1]
            
        if len(f) == 3:
            try:
                y = int(f[2])
            except:
                y = f[2]
                
        # now to run the func code
        if func == "rcv":
            try:
                new_val = inqueue.get(block=True, timeout=1.5)
                registers[x] = new_val
            except:
                print(f"no more messages left for Program {progam_id}")
                break
            rcv_msgs += 1
            #if verbose: print(f"Progam {progam_id} received msg {rcv_msgs}")
        elif func == "snd":
            with lock:
                x = registers[x] if type(x) is str else x
                outqueue.put(x)
                sent_msgs += 1
                if verbose: print(f"Program {progam_id} sent message {sent_msgs}")
        elif func == "set":
            y = registers[y] if type(y) is str else y
            registers[x] = y
        elif func == "add":
            y = registers[y] if type(y) is str else y
            registers[x] += y
        elif func == "mul":
            y = registers[y] if type(y) is str else y
            registers[x] = registers[x] * y
        elif func == "mod":
            y = registers[y] if type(y) is str else y
            registers[x] = registers[x] % y
        elif func == "jgz":
            x = registers[x] if type(x) is str else x
            if x > 0:
                i += registers[y] if type(y) is str else y
                continue # skip the i += step below
        else:
            print(f"found unknown command {func}, halting...")
            break
                
        i += 1
    
    with lock:
        print(f"----{progam_id} ends here----")
        print(f"P {progam_id} sent {sent_msgs} messages, received {rcv_msgs} msgs")
        print(registers)
        print("-------")
    
    return True

Testing the above on the test case:

In [9]:
test_input = """snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d""".strip().split("\n")
print(test_input)

q1 = Queue()
q2 = Queue()

t1 = Thread(target=do_ins_threaded, args=(test_input, 0, q1, q2, lock, False,))
t2 = Thread(target=do_ins_threaded, args=(test_input, 1, q2, q1, lock, False,))

t1.start() 
t2.start()

['snd 1', 'snd 2', 'snd p', 'rcv a', 'rcv b', 'rcv c', 'rcv d']
no more messages left for Program 1
----1 ends here----
P 1 sent 3 messages, received 3 msgs
defaultdict(<class 'int'>, {'p': 1, 'a': 1, 'b': 2, 'c': 0})
-------
no more messages left for Program 0
----0 ends here----
P 0 sent 3 messages, received 3 msgs
defaultdict(<class 'int'>, {'p': 0, 'a': 1, 'b': 2, 'c': 1})
-------


Now on the actual case:

In [17]:
p1_q = Queue()
p2_q = Queue()
#lock = Lock()

puzzle_thread_1 = Thread(target=do_ins_threaded, args=(puzzle_input, 0, p1_q, p2_q, lock, False,))
puzzle_thread_2 = Thread(target=do_ins_threaded, args=(puzzle_input, 1, p2_q, p1_q, lock, False,))

puzzle_thread_1.start() 
puzzle_thread_2.start()

no more messages left for Program 0
----0 ends here----
P 0 sent 7366 messages, received 7239 msgsno more messages left for Program 1

defaultdict(<class 'int'>, {'p': -91, 'i': 0, 'a': 26, 'b': 26, 'f': 0})
-------
----1 ends here----
P 1 sent 7239 messages, received 7366 msgs
defaultdict(<class 'int'>, {'p': -91, 'i': 0, 'a': 26, 'f': 0, 'b': 26})
-------


Eureka! The correct answer is `7239`. This took the longest time by far of all the advent puzzles.

Notes:

- learn more about threading and multiprocessing. 
- should I have used asyncio here?
- whats the best way to parse incoming commands in string form? use if/else statements, eval or something else?

In [18]:
p1_q.empty(), p2_q.empty(), puzzle_thread_1.is_alive(), puzzle_thread_2.is_alive()

(True, True, False, False)